In [1]:
import pandas as pd

from numpy import exp, log, pi, sin
from bokeh.plotting import figure, show
from bokeh.io import output_notebook
from sklearn.linear_model import LinearRegression

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
# Given values
Tbg = 2.8
Twarm = 298
Thot = 346
Tground = 291
Teff = 0.95 * Tground

In [4]:
# Load data from file
data = pd.read_csv('RRY100_2020_LAB1_elev_power.txt', names=['epsilon', 'Vant', 'Vwarm', 'Vhot'])

In [5]:
# Compute elevation in radians
data['epsilon_rad'] = data.epsilon * pi / 180

In [6]:
data['V1'] = data.Vwarm - data.Vant
data['V2'] = data.Vwarm - data.Vhot
data['V1_over_V2'] = data.V1 / data.V2
# Start with DThot = 0 and then adjust later on when fitting the data
DThot = -9.8783
data['Tant'] = Twarm - data.V1_over_V2 * (Twarm - (Thot + DThot))

In [7]:
data['M'] = 1 / sin(data.epsilon_rad)
data['M_tauZ'] = - log((Teff - data.Tant) / (Teff - Tbg))

In [8]:
# Plot M_tauZ as a function of M
p = figure(title='M(e) tauZ')
p.line(data.M, data.M_tauZ, line_width=2)
show(p)

In [9]:
# Fit a line for M_tauZ
model = LinearRegression().fit(data.M.to_numpy().reshape((-1, 1)), data.M_tauZ.to_numpy().reshape((-1, 1)))

# By adjusting delta T hot we should try to get the intercept as close to 0 as possible.
print(f'Intercept: {model.intercept_[0]}')
# Once that's done we can find tauZ which corresponds to the slope of the fitted line.
print(f'tauZ: {model.coef_[0][0]}')

Intercept: 3.589543890347713e-06
tauZ: 0.11272630463040384


In [10]:
# Tropospheric attenuation
data['trop_att'] = exp(data.M_tauZ)

# Tropospheric transmission
data['trop_tx'] = 1 / data.trop_att

In [11]:
# Compute T rec using the Y-factor
data['Y'] = data.Vhot / data.Vwarm
data['Trec'] = (Thot - data.Y * Twarm) / (data.Y - 1)

In [12]:
# Starting data and overall computed data
data

,epsilon,Vant,Vwarm,Vhot,epsilon_rad,V1,V2,V1_over_V2,Tant,M,M_tauZ,trop_att,trop_tx,Y,Trec
0,90.0,2.010751,2.447681,2.511143,1.570796,0.436930,-0.063461,-6.884993,35.532373,1.000000,0.127395,1.135865,0.880386,1.025927,1553.347003
1,85.0,2.009611,2.448699,2.510240,1.483530,0.439088,-0.061542,-7.134829,26.008176,1.003820,0.088623,1.092669,0.915190,1.025132,1611.890841
2,80.0,2.010059,2.448046,2.511598,1.396263,0.437987,-0.063551,-6.891861,35.270552,1.015427,0.126309,1.134632,0.881343,1.025960,1550.995100
3,75.0,2.009558,2.446263,2.511631,1.308997,0.436705,-0.065368,-6.680709,43.320007,1.035276,0.160254,1.173809,0.851928,1.026722,1498.301310
4,70.0,2.012279,2.448377,2.509254,1.221730,0.436098,-0.060877,-7.163560,24.912899,1.064178,0.084259,1.087911,0.919193,1.024864,1632.477775
5,65.0,2.022005,2.447956,2.510601,1.134464,0.425951,-0.062645,-6.799430,38.794153,1.103378,0.141026,1.151455,0.868466,1.025591,1577.675563
6,60.0,2.019001,2.447020,2.511884,1.047198,0.428019,-0.064864,-6.598699,46.446395,1.154701,0.173755,1.189764,0.840503,1.026507,1512.813435
7,57.5,2.021100,2.448152,2.510803,1.003564,0.427052,-0.062651,-6.816412,38.146788,1.185689,0.138306,1.148327,0.870832,1.025591,1577.661220
8,55.0,2.022050,2.447101,2.508814,0.959931,0.425051,-0.061713,-6.887571,35.434086,1.220775,0.126987,1.135402,0.880745,1.025219,1605.349528
9,52.5,2.024213,2.449590,2.508553,0.916298,0.425377,-0.058962,-7.214393,22.975077,1.260472,0.076585,1.079594,0.926274,1.024070,1696.161340
